In [10]:
import pickle
import pandas as pd
import gensim
from gensim.models import KeyedVectors
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

In [11]:
test_df = pd.read_csv('../../../lm-vol/LANL_test_unique.csv')

In [12]:
train_G = nx.read_graphml('../../../lm-vol/LANL_train.graphml')

In [13]:
test_G = nx.read_graphml('../../../lm-vol/LANL_test.graphml')

In [14]:
red_df = pd.read_csv('../../../lm-vol/redteam_complete.csv')
red_df = red_df.assign(src_user = red_df['source'].str.split('@').map(lambda x: x[0]))
red_df = red_df.assign(dest_user = red_df['destination'].str.split('@').map(lambda x: x[0]))
display(red_df)

,Unnamed: 0,time,source,source_computer,destination_computer,destination,authentication_type,logon_type,authentication_orientation,Success,src_user,dest_user
0,0,150885,U620@DOM1,C17693,C1003,U620@DOM1,NTLM,Network,LogOn,Success,U620,U620
1,1,151036,U748@DOM1,C17693,C305,U748@DOM1,NTLM,Network,LogOn,Success,U748,U748
2,2,151648,U748@DOM1,C17693,C728,U748@DOM1,NTLM,Network,LogOn,Success,U748,U748
3,3,151993,U6115@DOM1,C17693,C1173,U6115@DOM1,NTLM,Network,LogOn,Success,U6115,U6115
4,4,153792,U636@DOM1,C17693,C294,U636@DOM1,NTLM,Network,LogOn,Success,U636,U636
...,...,...,...,...,...,...,...,...,...,...,...,...
734,734,2552687,U655@DOM1,C17693,C828,U655@DOM1,NTLM,Network,LogOn,Success,U655,U655
735,735,2552687,U655@DOM1,C17693,C828,U655@DOM1,NTLM,Network,LogOn,Success,U655,U655
736,736,2552687,U655@DOM1,C17693,C828,U655@DOM1,NTLM,Network,LogOn,Success,U655,U655
737,737,2552687,U655@DOM1,C17693,C828,U655@DOM1,NTLM,Network,LogOn,Success,U655,U655


In [19]:
graph_structure = ('source', 'source_computer', 'destination_computer')
red_G = nx.Graph()
for index, row in red_df.iterrows():
    user_node = row[graph_structure[0]]
    ip_node = row[graph_structure[1]]
    service_node = row[graph_structure[2]]
    red_G.add_nodes_from([ip_node, service_node, user_node])
    red_G.add_edge(ip_node, user_node)
    red_G.add_edge(ip_node, service_node)

In [20]:
test_nodes = set(test_G.nodes)
train_nodes = set(train_G.nodes)
missing_nodes = test_nodes.difference(train_nodes)
print(len(test_nodes))
print(len(train_nodes))
print(len(missing_nodes))
filtered_test_G = test_G.copy()
for index, edge in enumerate(filtered_test_G.edges):
    if edge[0] in missing_nodes or edge[1] in missing_nodes:
        filtered_test_G.remove_edge(*edge)

100612
109240
6958


In [21]:
def operator_hadamard(u, v):
        return u * v
name = '2_28_NTML_Pikachu_500_epochs'
with open(f'../../../lm-vol/LANL_train_NTLM_Pikachu_filter_v2_lr.pkl', 'rb') as file:
    clf = pickle.load(file)
model = gensim.models.Word2Vec.load(f'../../../lm-vol/{name}_word2vec.model')
wv = KeyedVectors.load(f'../../../lm-vol/{name}_word2vec.wordvectors', mmap='r')
graph_structure = ('src_user', 'source_computer', 'dest_user')

In [22]:
red_edge_vectors = []
for index, edge in enumerate(filtered_test_G.edges):
    src = wv[edge[0]]
    dest = wv[edge[1]]
    src_to_dest = [operator_hadamard(src, dest)]
    if edge in red_G.edges or (edge[1], edge[0]) in red_G.edges:
        red_edge_vectors.append((src_to_dest, 1))
    else:
        red_edge_vectors.append((src_to_dest, 0))

KeyError: "Key 'ANONYMOUS LOGON@C586' not present"

In [ ]:
edge_vectors = []
for index, edge in enumerate(filtered_test_G.edges):
    src = wv[edge[0]]
    dest = wv[edge[1]]
    src_to_dest = [operator_hadamard(src, dest)]
    prob = clf.predict_proba(src_to_dest)
    if prob[0][1] <= 0.1:
        edge_vectors.append((src_to_dest, 1))
    else:
        edge_vectors.append((src_to_dest, 0))

In [8]:
anomalous_edge_vectors = filter(lambda x: x[1] == 1, edge_vectors)
red_edge_vector_filter = filter(lambda x: x[1] == 1, red_edge_vectors)

NameError: name 'edge_vectors' is not defined

In [ ]:
print(len(list(anomalous_edge_vectors)))
print(len(list(red_edge_vectors)))

In [ ]:
tsne = TSNE(n_components=2, random_state=7, perplexity=15)
embeddings = np.array([edge[0][0] for edge in edge_vectors])
embeddings_2d = tsne.fit_transform(embeddings)

In [ ]:
labels = np.array([edge[1] for edge in edge_vectors])
figure = plt.figure(figsize=(11, 9))

ax = figure.add_subplot(111)

ax.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], s=0.5, c=labels, cmap='bwr')

In [ ]:
red_labels = np.array([edge[1] for edge in red_edge_vectors])
figure = plt.figure(figsize=(11, 9))

ax = figure.add_subplot(111)

ax.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], s=0.5, c=red_labels, cmap='bwr')

In [ ]:
true_positives = 0
false_positives = 0
true_negatives = 0
false_negatives = 0
for index, label in enumerate(labels):
    if label == 1:
        if red_labels[index] == 1:
            true_positives += 1
        else:
            false_positives += 1
    else:
        if red_labels[index] == 1:
            false_negatives += 1
        else:
            true_negatives += 1
print(f'total test edges: {len(test_G.edges)}')
print(f'valid test edges: {len(labels)}')
print(f'missing test edges: {len(test_G.edges)-len(labels)}')
print(f'true postives: {true_positives}')
print(f'true negatives: {true_negatives}')
print(f'false postives: {false_positives}')
print(f'false negatives: {false_negatives}')
print(f'false positive rate: {false_positives/(true_negatives+false_positives)}')
print(f'true positive rate: {true_positives/(true_positives+false_negatives)}')

In [ ]:
anomalous_edge_vectors = filter(lambda x: x[1] == 1, edge_vectors)

In [ ]:
anomaly_edge = None
max_iter = 10000000
shortest_paths = []
for index, edge in enumerate(filtered_test_G.edges):
    if labels[index] == 1:
        anomaly_edge = edge
        src_neighbors = [n for n in train_G.neighbors(anomaly_edge[0])]
        dest_neighbors = [n for n in train_G.neighbors(anomaly_edge[1])]
#         print(f'source {anomaly_edge[0]} has {len(src_neighbors)} neighbors')
#         print(f'destination {anomaly_edge[1]} has {len(dest_neighbors)} neighbors')
        try:
            path = nx.shortest_path(train_G, source=anomaly_edge[0], target=anomaly_edge[1])
#             print(f'shorest path: {path}')
        except:
            path = []
#             print('----------------no shortest path---------------------------')
        shortest_paths.append(path)

In [ ]:
count = 0
null_count = 0
for path in shortest_paths:
    if len(path) >=4:
        count +=1
    elif len(path) == 0:
        null_count +=1
print(count)
print(len(shortest_paths))
print(null_count)

In [ ]:
nx.number_connected_components(train_G)